## 1.파일 로드

In [1]:
import pandas as pd

session = pd.read_excel('data/raw_data/raw_session_data.xlsx')
activity = pd.read_excel('data/raw_data/raw_activity_data.xlsx')
product = pd.read_excel('data/raw_data/raw_product_data.xlsx')

## 2.세션 테이블 정제

In [2]:
session.head()

,userId,sessionId,sessionDate,device,platform
0,689679556_1578719054,1580440083,2020-01-31,desktop,Windows
1,689679556_1578719054,1580193439,2020-01-28,desktop,Windows
2,689679556_1578719054,1578719060,2020-01-11,desktop,Windows
3,689679556_1578719054,1578719056,2020-01-11,desktop,Windows
4,475325885_1578657870,1578833601,2020-01-12,mobile,iOS


In [3]:
session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11888 entries, 0 to 11887
Data columns (total 5 columns):
userId         11888 non-null object
sessionId      11888 non-null int64
sessionDate    11888 non-null object
device         11888 non-null object
platform       11888 non-null object
dtypes: int64(1), object(4)
memory usage: 464.5+ KB


### 1)sessionDate 날짜 타입으로 바꾸기

In [4]:
session['sessionDate'] = pd.to_datetime(session['sessionDate'])

In [5]:
session.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11888 entries, 0 to 11887
Data columns (total 5 columns):
userId         11888 non-null object
sessionId      11888 non-null int64
sessionDate    11888 non-null datetime64[ns]
device         11888 non-null object
platform       11888 non-null object
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 464.5+ KB


## 2. 활동 테이블

### 1)transectionId 형식 통일

In [6]:
activity['transectionId'].value_counts()

20200121-0000369     2
20200131-0000687     2
Pay-1578720495668    2
Pay-1579829922452    1
20200128-0000108     1
                    ..
20200114-0000091     1
20200130-0000083     1
Pay-1579435190193    1
20200128-0000196     1
20200124-0000675     1
Name: transectionId, Length: 2603, dtype: int64

아직까지 구분점을 찾지 못했음. 일단 진행

### 2)activity Time 형식 변경

In [9]:
def extract_activityTime(data):
    
    if pd.isnull(data):
        return data
    
    else:
        
        tmp_data = data.split('.')[0]
        result = tmp_data.split('T')
        result = result[0] + ' ' + result[1]
        
    return result

In [10]:
activity['activityTime'] = activity['activityTime'].apply(extract_activityTime)

In [11]:
activity['activityTime'] = pd.to_datetime(activity['activityTime'])

activity['activityTime'].head()

0   2020-01-11 14:28:15
1   2020-01-10 21:11:15
2   2020-01-08 02:03:36
3   2020-01-26 09:18:25
4   2020-01-30 18:51:18
Name: activityTime, dtype: datetime64[ns]

### 3)source 범주 통일

In [12]:
activity['source'].value_counts()

naver                         1135
facebook                       469
(direct)                       447
instagram.com                  291
google                          74
review6.cre.ma                  36
facebook.com                    21
m.shopping.naver.com            20
castbox.shopping.naver.com      15
IGShopping                      15
m.pay.naver.com                 13
m.naver.com                     10
naver.com                        7
m.blog.naver.com                 7
daum                             7
m.keep.naver.com                 6
tmpl.co.kr                       6
cr2.shopping.naver.com           4
order.pay.naver.com              4
m.cafe.naver.com                 3
pmon.navercorp.com               2
shopping.naver.com               2
cre.ma                           2
cafe.naver.com                   2
m.tmpl.co.kr                     2
m.bookmark.naver.com             1
daum.net                         1
ksmobile.inicis.com              1
mup.mobilians.co.kr 

- naver
- facebook
- direct
- instagram: igshopping 포함
- google
- crema: 크리마 사이트(cre.ma)
- tmpl: 자사 홈피
- daum
- etc: bing, 확인불명 사이트 등등

In [13]:
def clean_source(source):
    if pd.isnull(source):
        return np.nan
    
    if 'naver' in source:
        return 'naver'
    elif 'facebook' in source:
        return 'facebook'
    elif 'direct' in source:
        return 'direct'
    elif 'instagram' in source:
        return 'instagram'
    elif 'IGshopping' in source:
        return 'instagram'
    elif 'google' in source:
        return 'google'
    elif 'cre.ma' in source:
        return 'crema'
    elif 'tmpl' in source:
        return 'tmpl'
    elif 'daum' in source:
        return 'daum'
    else:
        return 'etc'

In [14]:
activity['source'] = activity['source'].apply(clean_source)

activity['source'].value_counts()

naver        1232
facebook      490
direct        447
instagram     291
google         74
crema          38
etc            18
daum            8
tmpl            8
Name: source, dtype: int64

### 4)medium 범주 통일

In [15]:
activity['medium'].value_counts()

cpc         644
organic     573
referral    458
(none)      447
social      378
display      91
Social       15
Name: medium, dtype: int64

In [16]:
activity['medium'] = activity['medium'].str.lower()
activity['medium'].value_counts()

cpc         644
organic     573
referral    458
(none)      447
social      393
display      91
Name: medium, dtype: int64

### 5)transectionRevenue 타입 변경

In [17]:
activity['transectionRevenue'] = pd.to_numeric(activity['transectionRevenue'])

activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606 entries, 0 to 2605
Data columns (total 12 columns):
userId                2606 non-null object
sessionId             2606 non-null int64
transectionId         2606 non-null object
activityTime          2606 non-null datetime64[ns]
source                2606 non-null object
medium                2606 non-null object
channelGrouping       2606 non-null object
campaign              2606 non-null object
keyword               2606 non-null object
hostname              2606 non-null object
landingPagePath       2606 non-null object
transectionRevenue    2606 non-null int64
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 244.4+ KB


## 3. 저장

In [37]:
session.to_excel('data/raw_data/session_data_v1.xlsx',index = False)
activity.to_excel('data/raw_data/activity_data_v1.xlsx',index = False)